In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [162]:
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_movies, k=30):
        super().__init__()
        self.U = nn.Embedding(n_users, k)
        self.M = nn.Embedding(n_movies, k)
        #initialize embedding matrices
        torch.nn.init.kaiming_normal_(self.U.weight.data)
        torch.nn.init.kaiming_normal_(self.M.weight.data)
    
    def forward(self, usr_idx, mov_idx):
        return (self.U(usr_idx) * self.M(mov_idx)).sum(1)

In [200]:
def get_emb(ni,nf):
    e = nn.Embedding(ni, nf)
    e.weight.data.uniform_(-0.01,0.01)
    return e

class EmbeddingDotBias(nn.Module):
    def __init__(self, n_users, n_movies, n_factors):
        super().__init__()
        (self.u, self.m, self.ub, self.mb) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors), (n_users,1), (n_movies,1)
        ]]
        
    def forward(self, users, movies):
        um = (self.u(users)* self.m(movies)).sum(1)
        res = um + self.ub(users).squeeze() + self.mb(movies).squeeze()
#        res = torch.sigmoid(res) * (9-1) + 1
        return res.view(-1, 1)

In [201]:
test = np.array([
    [4, 2, -1, -1],
    [6, 9, -1, 1],
    [1, -1, 3, -1],
    [5, -1, -1, 4],
    [-1, 3, 4, 5],
])

In [196]:
model = MatrixFactorization(test.shape[0], test.shape[1], 3)

In [208]:
model = EmbeddingDotBias(test.shape[0], test.shape[1], 3)

In [209]:
l = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-2)

In [210]:
Z = np.argwhere(test > -1)
epochs = 100


for e in range(epochs):
    for i, j in Z:
      #need to use variables for autograd / loss function
        m_ij = Variable(torch.FloatTensor([test[i,j]]))
        i = Variable(torch.LongTensor([i]))
        j = Variable(torch.LongTensor([j]))

        m_ij_hat = model(i,j)
        loss = l(m_ij, m_ij_hat)

        #backprop
        loss.backward()
        optimizer.step()

In [211]:
model(Variable(torch.LongTensor([0])),Variable(torch.LongTensor([0])))

tensor([[-1652.2900]], grad_fn=<ViewBackward>)

In [212]:
for i in range(test.shape[0]):
    for j in range(test.shape[1]):
        print(str(model(Variable(torch.LongTensor([i])),Variable(torch.LongTensor([j]))).data[0]) + ' ', end='')
    print('')

tensor([-1652.2900]) tensor([-25.7033]) tensor([54.2778]) tensor([2.0120]) 
tensor([-1197.5562]) tensor([-0.8996]) tensor([49.6696]) tensor([-13.8453]) 
tensor([1791.9508]) tensor([5.2046]) tensor([-67.6159]) tensor([-9.6897]) 
tensor([-16872.3398]) tensor([150.5946]) tensor([663.4835]) tensor([-239.5280]) 
tensor([-264.5982]) tensor([-0.7314]) tensor([1.5184]) tensor([7.4993]) 


In [261]:
import numpy as np
from scipy.sparse import rand as sprand
import torch
from torch.autograd import Variable

# Make up some random explicit feedback ratings
# and convert to a numpy array
n_users = 10
n_items = 10
ratings = np.random.rand(n_users, n_items)

In [270]:
model= EmbeddingDotBias(ratings.shape[0], ratings.shape[1], 100)
l = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)#, weight_decay=1e-2)
optimizer = torch.optim.Adagrad(model.parameters(), lr=.1)

In [272]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]
epochs = 30

for e in range(epochs):
    for row, col in zip(*(rows, cols)):
        # Turn data into variables
        rating = Variable(torch.FloatTensor([ratings[row, col]]))
        row = Variable(torch.LongTensor([np.long(row)]))
        col = Variable(torch.LongTensor([np.long(col)]))

        # Predict and calculate loss
        prediction = model(row, col)
        loss = l(prediction, rating)

        # Backpropagate
        loss.backward()

        # Update the parameters
        optimizer.step()

In [273]:
for i in range(5):
    print(ratings[i,:5])

[0.39623922 0.85700251 0.02117972 0.05625527 0.83430948]
[0.44202626 0.27656302 0.37642793 0.42788834 0.46807313]
[0.98306338 0.9753429  0.80496421 0.87571331 0.29794588]
[0.64034244 0.59277837 0.16859842 0.79248708 0.75846907]
[0.16666699 0.03781352 0.69918887 0.78551424 0.29208783]


In [274]:
for i in range(5):
    row = Variable(torch.LongTensor([np.long(i)]))
    for j in range(5):
        col = Variable(torch.LongTensor([np.long(j)]))
        print(str(model(row, col).data[0]) + " ", end='')
    print()

tensor([-5.2283]) tensor([23.6450]) tensor([2.4460]) tensor([1.0319]) tensor([8.1186]) 
tensor([-0.1754]) tensor([-9.6513]) tensor([-0.3928]) tensor([1.4186]) tensor([21.7434]) 
tensor([5.2027]) tensor([-12.1442]) tensor([2.7429]) tensor([-2.3188]) tensor([1.1454]) 
tensor([-6.4635]) tensor([8.9304]) tensor([1.6601]) tensor([-5.0688]) tensor([-1.9566]) 
tensor([-2.7202]) tensor([15.9583]) tensor([1.3614]) tensor([-1.0350]) tensor([-5.5708]) 
